**Step 1: Install all the required dependencies**

In [ ]:
%%capture
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install soundfile
!pip install jiwer
!pip install torchaudio

**Step 2: Preparing the vocabularing and processing the dataset**

In [ ]:
# Ignore if not working on google colab
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
# Refer to Swahili_data_sample.csv in this reposity to check how the data must look like. Prepare two csv files - one for train
# and one for test set. 

from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': ["/content/gdrive/MyDrive/Swahili-ASR/SwahiliDataTrain.csv"],
                                              'test': '/content/gdrive/MyDrive/Swahili-ASR/SwahiliDataTest.csv'})

In [ ]:
datasetTrain = dataset['train']
datasetTrain = datasetTrain.remove_columns(['Unnamed: 0'])
datasetTest = dataset['test']
datasetTest = datasetTest.remove_columns(['Unnamed: 0'])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(datasetTrain.remove_columns(["path"]))

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["transcript"] = re.sub(chars_to_ignore_regex, '', batch["transcript"]).lower() + " "
    return batch

datasetTrain = datasetTrain.map(remove_special_characters)
datasetTest = datasetTest.map(remove_special_characters)

In [ ]:
len(datasetTrain)

In [ ]:
print(show_random_elements(datasetTrain.remove_columns(["path"])))

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcript"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = datasetTrain.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=datasetTrain.column_names)
vocab_test = datasetTest.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=datasetTest.column_names)

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-swahili-demo")

In [ ]:
datasetTrain[0]

In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
  
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["transcript"]
    return batch



In [ ]:
datasetTest = datasetTest.map(speech_file_to_array_fn, remove_columns=datasetTest.column_names)

In [ ]:
len(datasetTest)

In [ ]:
datasetTrain = datasetTrain.map(speech_file_to_array_fn, remove_columns=datasetTrain.column_names)


In [ ]:
## Important - If the data you are using is not 16KHz, make changes to this cell accordingly to resample the data to 16KHz

# import librosa
# import numpy as np

# def resample(batch):
#     batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
#     batch["sampling_rate"] = 16_000
#     return batch

# datasetTrain = datasetTrain.map(resample, num_proc=4)
# datasetTest = datasetTest.map(resample, num_proc=4)

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(datasetTrain)-1)

ipd.Audio(data=np.asarray(datasetTrain[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
rand_int = random.randint(0, len(datasetTrain)-1)

print("Target text:", datasetTrain[rand_int]["target_text"])
print("Input array shape:", np.asarray(datasetTrain[rand_int]["speech"]).shape)
print("Sampling rate:", datasetTrain[rand_int]["sampling_rate"])

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
datasetTrain = datasetTrain.map(prepare_dataset, remove_columns=datasetTrain.column_names, batch_size=8, num_proc=4, batched=True)
datasetTest= datasetTest.map(prepare_dataset, remove_columns=datasetTest.column_names, batch_size=8, num_proc=4, batched=True)

**Step 3: Prepare the models for training**

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-swahili-demo4",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=datasetTrain,
    eval_dataset=datasetTest,
    tokenizer=processor.feature_extractor,
)

**Step 4: Start Training!**

In [ ]:
trainer.train()

**Step 5: Checking the validity on test set**

In [ ]:
sample_number = 0 # choose any sample from the test set where sample_number is the index

from transformers import Wav2Vec2ForCTC
model = Wav2Vec2ForCTC.from_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-swahili-demo3/checkpoint-200/").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-swahili-demo")

input_dict = processor(datasetTest[sample_number]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

dataset = load_dataset('csv', data_files={'train': ["/content/gdrive/MyDrive/Swahili-ASR/SwahiliDataTrain.csv"],
                                              'test': '/content/gdrive/MyDrive/Swahili-ASR/SwahiliDataTest.csv'})

datasetTrain_demo = dataset['train']
datasetTrain_demo = datasetTrain_demo.remove_columns(['Unnamed: 0'])
datasetTest_demo = dataset['test']
datasetTest_demo = datasetTest_demo.remove_columns(['Unnamed: 0'])

print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(datasetTest[sample_number]["transcript"].lower())